In [1]:
import numpy as np
import pandas as pd
import spacy
import nltk, re
from nltk.corpus import stopwords
nltk.download('stopwords')

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /home/ideis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df_ratings = pd.read_csv('data/movies/ratings.csv')
df_movies = pd.read_csv('data/movies/movies.csv')
df_links = pd.read_csv('data/movies/links.csv')

In [3]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

def clean_text(text, remove_stopwords=True):
    '''Clean the text, with the option to remove stopwords'''
    text = text.lower()
    tokens = tokenizer.tokenize(text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        tokens = [w for w in tokens if not w in stops]
    
    text = " ".join(tokens)
    return tokens

def process_title(title): 
    # strip away numbers and parenthesis
    title = title.replace('(','').replace(')','')
    title = re.sub(r'\d+','',title)
    # strip away "part" word
    title = re.sub(r'[Pp]art','',title)
    #strip II and III and IV
    title = title.replace('II','').replace('III','').replace('IV','')
    return title

df_movies['title'] = df_movies['title'].apply(process_title) 

In [4]:
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner', 'textcat'])

In [5]:
def get_vector(doc):
    return doc.vector

df_movies['docs'] = df_movies['title'].apply(nlp)
df_movies['doc2vec'] = df_movies['docs'].apply(get_vector)
df_movies.drop(['docs'], axis=1, inplace=True)
df_movies.to_csv('data/movies/movies_titles_with_doc2vec.csv')

In [6]:
# Create links to imdb website
def make_urls(df_links):
    df_links['pagePath'] = 'https://www.imdb.com/title/tt00' + df_links['imdbId'].astype(str)
    df_links.drop(['imdbId'],axis=1,inplace=True)
    df_links.drop(['tmdbId'],axis=1,inplace=True)
    return df_links

df_links = make_urls(df_links)

In [9]:
# Merge by userId all csv files
def merge_df(df_ratings, df_links, df_movies):
    df_result = pd.merge(df_ratings, df_links, on='movieId', how='left')
    df_result = pd.merge(df_result, df_movies, on='movieId', how='left')
    df_result['visitStartTime'] = df_result['timestamp']
    df_result.set_index(['userId', 'movieId'], inplace=True)
    df_result = df_result[['visitStartTime', 'pagePath', 'title', 'genres', 'rating', 'doc2vec']]
    return df_result

df_result = merge_df(df_ratings, df_links, df_movies)
df_result.to_csv('data/movies/movies_final_dataset.csv')

In [10]:
display(df_result.info())
display(df_result.head(20))
display(df_result.tail(20))

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 100004 entries, (1, 31) to (671, 6565)
Data columns (total 6 columns):
visitStartTime    100004 non-null int64
pagePath          100004 non-null object
title             100004 non-null object
genres            100004 non-null object
rating            100004 non-null float64
doc2vec           100004 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 5.0+ MB


None

visitStartTime                               pagePath  \
userId movieId                                                          
1      31           1260759144  https://www.imdb.com/title/tt00112792   
       1029         1260759179   https://www.imdb.com/title/tt0033563   
       1061         1260759182  https://www.imdb.com/title/tt00117665   
       1129         1260759185   https://www.imdb.com/title/tt0082340   
       1172         1260759205   https://www.imdb.com/title/tt0095765   
       1263         1260759151   https://www.imdb.com/title/tt0077416   
       1287         1260759187   https://www.imdb.com/title/tt0052618   
       1293         1260759148   https://www.imdb.com/title/tt0083987   
       1339         1260759125  https://www.imdb.com/title/tt00103874   
       1343         1260759131  https://www.imdb.com/title/tt00101540   
       1371         1260759135   https://www.imdb.com/title/tt0079945   
       1405         1260759203  https://www.imdb.com/title/tt00115641   
       1953         1260759191   https://www.imdb.com/title/tt0067116   
       2105         1260759139   https://www.imdb.com/title/tt0084827   
       2150         1260759194   https://www.imdb.com/title/tt0080801   
       2193         1260759198   https://www.imdb.com/title/tt0096446   
       2294         1260759108  https://www.imdb.com/title/tt00120587   
       2455         1260759113   https://www.imdb.com/title/tt0091064   
       2968         1260759200   https://www.imdb.com/title/tt0081633   
       3671         1260759117   https://www.imdb.com/title/tt0071230   

                                                 title  \
userId movieId                                           
1      31                             Dangerous Minds    
       1029                                     Dumbo    
       1061                                  Sleepers    
       1129                      Escape from New York    
       1172     Cinema Paradiso Nuovo cinema Paradiso    
       1263                          Deer Hunter, The    
       1287                                   Ben-Hur    
       1293                                    Gandhi    
       1339             Dracula Bram Stoker's Dracula    
       1343                                 Cape Fear    
       1371             Star Trek: The Motion Picture    
       1405           Beavis and Butt-Head Do America    
       1953                    French Connection, The    
       2105                                      Tron    
       2150                   Gods Must Be Crazy, The    
       2193                                    Willow    
       2294                                      Antz    
       2455                                  Fly, The    
       2968                              Time Bandits    
       3671                           Blazing Saddles    

                                                     genres  rating  \
userId movieId                                                        
1      31                                             Drama     2.5   
       1029                Animation|Children|Drama|Musical     3.0   
       1061                                        Thriller     3.0   
       1129                Action|Adventure|Sci-Fi|Thriller     2.0   
       1172                                           Drama     4.0   
       1263                                       Drama|War     2.0   
       1287                          Action|Adventure|Drama     2.0   
       1293                                           Drama     2.0   
       1339                 Fantasy|Horror|Romance|Thriller     3.5   
       1343                                        Thriller     2.0   
       1371                                Adventure|Sci-Fi     2.5   
       1405                Adventure|Animation|Comedy|Crime     1.0   
       1953                           Action|Crime|Thriller     4.0   
       2105                         Action|Adventure|Sci-Fi     4.0

visitStartTime                               pagePath  \
userId movieId                                                          
671    5010         1066793004  https://www.imdb.com/title/tt00265086   
       5218         1065111990  https://www.imdb.com/title/tt00268380   
       5299         1065112004  https://www.imdb.com/title/tt00259446   
       5349         1065111863  https://www.imdb.com/title/tt00145487   
       5377         1064245557  https://www.imdb.com/title/tt00276751   
       5445         1064891627  https://www.imdb.com/title/tt00181689   
       5464         1064891549  https://www.imdb.com/title/tt00257044   
       5669         1063502711  https://www.imdb.com/title/tt00310793   
       5816         1065111963  https://www.imdb.com/title/tt00295297   
       5902         1064245507  https://www.imdb.com/title/tt00268126   
       5952         1063502716  https://www.imdb.com/title/tt00167261   
       5989         1064890625  https://www.imdb.com/title/tt00264464   
       5991         1064245387  https://www.imdb.com/title/tt00299658   
       5995         1066793014  https://www.imdb.com/title/tt00253474   
       6212         1065149436  https://www.imdb.com/title/tt00305669   
       6268         1065579370  https://www.imdb.com/title/tt00316188   
       6269         1065149201  https://www.imdb.com/title/tt00334416   
       6365         1070940363  https://www.imdb.com/title/tt00234215   
       6385         1070979663  https://www.imdb.com/title/tt00298228   
       6565         1074784724  https://www.imdb.com/title/tt00329575   

                                                   title  \
userId movieId                                             
671    5010                             Black Hawk Down    
       5218                                     Ice Age    
       5299                    My Big Fat Greek Wedding    
       5349                                  Spider-Man    
       5377                                 About a Boy    
       5445                             Minority Report    
       5464                           Road to Perdition    
       5669                       Bowling for Columbine    
       5816     Harry Potter and the Chamber of Secrets    
       5902                                  Adaptation    
       5952      Lord of the Rings: The Two Towers, The    
       5989                         Catch Me If You Can    
       5991                                     Chicago    
       5995                                Pianist, The    
       6212                     Bringing Down the House    
       6268                       Raising Victor Vargas    
       6269                                      Stevie    
       6365                        Matrix Reloaded, The    
       6385                                 Whale Rider    
       6565                                  Seabiscuit    

                                               genres  rating  \
userId movieId                                                  
671    5010                          Action|Drama|War     2.0   
       5218       Adventure|Animation|Children|Comedy     2.0   
       5299                            Comedy|Romance     3.0   
       5349          Action|Adventure|Sci-Fi|Thriller     4.0   
       5377                      Comedy|Drama|Romance     4.0   
       5445      Action|Crime|Mystery|Sci-Fi|Thriller     4.5   
       5464                               Crime|Drama     3.0   
       5669                               Documentary     4.0   
       5816                         Adventure|Fantasy     4.0   
       5902                      Comedy|Drama|Romance     3.5   
       5952                         Adventure|Fantasy     5.0   
       5989                               Crime|Drama     4.0   
       5991                Comedy|Crime|Drama|Musical     4.5   
       5995                                 Drama|War     4.0   
       6212                                    C